In [2]:
import pandas as pd
import numpy as np
import re

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import pyconll
import nltk
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger, RegexpTagger

import warnings
warnings.filterwarnings("ignore")

In [44]:
import urllib.request
url = 'https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu'
filename = 'ru_syntagrus-ud-train-a.conllu'
urllib.request.urlretrieve(url, filename)

('ru_syntagrus-ud-train-a.conllu', <http.client.HTTPMessage at 0x27a05751c50>)

In [45]:
url = 'https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu'
filename = 'ru_syntagrus-ud-dev.conllu'
urllib.request.urlretrieve(url, filename)

('ru_syntagrus-ud-dev.conllu', <http.client.HTTPMessage at 0x27a057045d0>)

In [47]:
data_train = pyconll.load_from_file('ru_syntagrus-ud-train-a.conllu')
data_test = pyconll.load_from_file('ru_syntagrus-ud-dev.conllu')

In [48]:
fdata_train = []
for sent in data_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in data_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in data_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [49]:
len(fdata_train), len(fdata_test), len(fdata_sent_test)

(24516, 8906, 8906)

In [53]:
fdata_train

[[('Анкета', 'NOUN'), ('.', 'PUNCT')],
 [('Начальник', 'NOUN'),
  ('областного', 'ADJ'),
  ('управления', 'NOUN'),
  ('связи', 'NOUN'),
  ('Семен', 'PROPN'),
  ('Еремеевич', 'PROPN'),
  ('был', 'AUX'),
  ('человек', 'NOUN'),
  ('простой', 'ADJ'),
  (',', 'PUNCT'),
  ('приходил', 'VERB'),
  ('на', 'ADP'),
  ('работу', 'NOUN'),
  ('всегда', 'ADV'),
  ('вовремя', 'ADV'),
  (',', 'PUNCT'),
  ('здоровался', 'VERB'),
  ('с', 'ADP'),
  ('секретаршей', 'NOUN'),
  ('за', 'ADP'),
  ('руку', 'NOUN'),
  ('и', 'CCONJ'),
  ('иногда', 'ADV'),
  ('даже', 'PART'),
  ('писал', 'VERB'),
  ('в', 'ADP'),
  ('стенгазету', 'NOUN'),
  ('заметки', 'NOUN'),
  ('под', 'ADP'),
  ('псевдонимом', 'NOUN'),
  ('"', 'PUNCT'),
  ('Муха', 'NOUN'),
  ('"', 'PUNCT'),
  ('.', 'PUNCT')],
 [('В', 'ADP'),
  ('приемной', 'NOUN'),
  ('его', 'PRON'),
  ('с', 'ADP'),
  ('утра', 'NOUN'),
  ('ожидали', 'VERB'),
  ('посетители', 'NOUN'),
  (',', 'PUNCT'),
  ('-', 'PUNCT'),
  ('кое-кто', 'PRON'),
  ('с', 'ADP'),
  ('важными', 'ADJ'),

In [54]:
default_tagger = nltk.DefaultTagger('NN')
default_acc = default_tagger.evaluate(fdata_test)

unigram_tagger = UnigramTagger(fdata_train)
unigram_acc = unigram_tagger.evaluate(fdata_test)

bigram_tagger = BigramTagger(fdata_train)
bigram_acc = bigram_tagger.evaluate(fdata_test)

trigram_tagger = TrigramTagger(fdata_train)
trigram_acc = trigram_tagger.evaluate(fdata_test)

bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
bigram_unigram_acc = bigram_tagger.evaluate(fdata_test)

trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
trigram_bigram_unigram_acc = trigram_tagger.evaluate(fdata_test)

print(f'Accuracy:\nDefault Tagger: {round(default_acc, 3)},\nUnigram Tagger: {round(unigram_acc, 3)},\nBigram Tagger: {round(bigram_acc, 5)},\n'
      f'Trigram Tagger: {round(trigram_acc, 3)},\nBigram and Unigram Tagger: {round(bigram_unigram_acc, 5)},\n'
      f'Trigram, Bigram and Unigram Tagger: {round(trigram_bigram_unigram_acc, 5)},\n')

Accuracy:
Default Tagger: 0.0,
Unigram Tagger: 0.824,
Bigram Tagger: 0.60939,
Trigram Tagger: 0.178,
Bigram and Unigram Tagger: 0.82928,
Trigram, Bigram and Unigram Tagger: 0.82914,



In [55]:
def union_taggers(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NN') 
tag = union_taggers(fdata_train,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.evaluate(fdata_test) 

0.827905462595221

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.preprocessing import LabelEncoder

In [57]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(' ' if tok[0] is None else tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [58]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
train_enc_labels

array([ 7, 13,  7, ...,  1, 11, 13], dtype=int64)

In [59]:
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [61]:
test_enc_labels = le.transform(test_label)
test_enc_labels

array([ 7, 13,  1, ...,  0,  7, 13])

In [62]:
%time

vectorizers = [CountVectorizer(ngram_range=(1, 5), analyzer='char'), 
               TfidfVectorizer(ngram_range=(1, 5), analyzer='char'), 
               HashingVectorizer(ngram_range=(1, 5), analyzer='char', n_features=1000)] 
vectorizers_word = [CountVectorizer(ngram_range=(1, 5), analyzer='word'), 
               TfidfVectorizer(ngram_range=(1, 5), analyzer='word'), 
               HashingVectorizer(ngram_range=(1, 5), analyzer='word', n_features=1000)] 
n_features = [2000, 3000, 5000]
vectorizers_hash = [HashingVectorizer(ngram_range=(1, 5), analyzer='char', n_features=feat) for feat in n_features]
vectorizers_hash_word = [HashingVectorizer(ngram_range=(1, 5), analyzer='word', n_features=feat) for feat in n_features]
f1_scores = []
accuracy_scores = []

for vectorizer in vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word:
    X_train = vectorizer.fit_transform(train_tok)
    X_test = vectorizer.transform(test_tok)
    
    lr = LogisticRegression(random_state=0, max_iter=100)
    lr.fit(X_train, train_enc_labels)
    pred = lr.predict(X_test)
    f1 = f1_score(test_enc_labels, pred, average='weighted')
    f1_scores.append(f1)
    acc = accuracy_score(test_enc_labels, pred)
    accuracy_scores.append(acc)
    
    print(vectorizer)
    print(classification_report(test_enc_labels, pred, target_names=le.classes_))

CPU times: total: 0 ns
Wall time: 0 ns
CountVectorizer(analyzer='char', ngram_range=(1, 5))
              precision    recall  f1-score   support

         ADJ       0.94      0.92      0.93     15103
         ADP       0.98      1.00      0.99     13717
         ADV       0.91      0.93      0.92      7783
         AUX       0.82      0.96      0.88      1390
       CCONJ       0.89      0.97      0.93      5672
         DET       0.83      0.79      0.81      4265
        INTJ       0.39      0.29      0.33        24
        NOUN       0.94      0.97      0.96     36238
      NO_TAG       1.00      0.77      0.87       265
         NUM       0.86      0.89      0.88      1734
        PART       0.95      0.77      0.85      5125
        PRON       0.90      0.84      0.87      7444
       PROPN       0.84      0.66      0.73      5473
       PUNCT       1.00      1.00      1.00     29186
       SCONJ       0.75      0.97      0.85      2865
         SYM       1.00      0.85      0.92

In [64]:
import nltk
from nltk.tokenize import word_tokenize
import matplotlib
import pyconll
import corus
from corus import load_ne5
import re
import spacy
from spacy import displacy

In [75]:
import nltk
import os
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\Пользователь\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Пользователь\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Пользователь\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Пользователь\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [66]:
nltk.help.upenn_tagset('RB')
nltk.help.upenn_tagset('NN')
nltk.help.upenn_tagset('VB')

RB: adverb
    occasionally unabatingly maddeningly adventurously professedly
    stirringly prominently technologically magisterially predominately
    swiftly fiscally pitilessly ...
NN: noun, common, singular or mass
    common-carrier cabbage knuckle-duster Casino afghan shed thermostat
    investment slide humour falloff slick wind hyena override subhumanity
    machinist ...
VB: verb, base form
    ask assemble assess assign assume atone attention avoid bake balkanize
    bank begin behold believe bend benefit bevel beware bless boil bomb
    boost brace break bring broil brush build ...


In [77]:
dataset_for_nltk = []

for filename in os.scandir("collection5/"):
    if filename.is_file():
        name, ext = os.path.splitext(filename.path)
        if ext == ".txt":
          with(open(filename.path,"r",encoding='utf-8') as f):
            text = f.read()
            tokens = text.split()
          
          tagged_tokens = {}

          with(open(name + ".ann","r",encoding='utf-8') as f):
            for line in f:
              parts = line.split()
              tag = parts[1]
              name = ' '.join(parts[4:])
              tagged_tokens[name] = tag

          dataset_for_nltk.append((text, tagged_tokens))

In [79]:
for entry in dataset_for_nltk:
  pred = [(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(entry[0]))) if hasattr(chunk, 'label')]
  print(entry[1])
  print(pred)
  break

{'Россия': 'GEOPOLIT', 'США': 'GEOPOLIT', 'Грузию': 'GEOPOLIT', 'МОСКВА': 'LOC', 'РИА Новости': 'MEDIA', 'Тбилиси': 'GEOPOLIT', 'России': 'GEOPOLIT', 'Григорий Карасин': 'PER', 'Дэниэлом Фридом': 'PER', 'Южной Осетии': 'GEOPOLIT', 'Вашингтона': 'GEOPOLIT', 'МИД': 'ORG'}
[('Россия', 'PERSON'), ('МОСКВА', 'ORGANIZATION'), ('РИА Новости', 'ORGANIZATION'), ('Россия', 'PERSON'), ('Тбилиси', 'PERSON'), ('России Григорий Карасин', 'PERSON'), ('Тбилиси', 'PERSON'), ('МИД России', 'ORGANIZATION')]
